# Problem Set 4

## Question 2

In this exercise, we numerically study the behaviors of the wild and the pairs bootstrap by considering the following model:
\begin{align}\label{Q4}\tag{1}
Y_i=Z_{1i} + Z_{2i} + Z_{3i} + \varrho_0 Z_{1i} Z_{2i} + (1+\lambda_0 Z_{1i})\epsilon_i~,
\end{align}
for $i=1,\ldots, n$. The sample $\{Y_i,Z_{1i},Z_{2i},Z_{3i}\}_{i=1}^n$ is generated as follows.

(1) Generate $V_i,Z_{2i}$ and $Z_{3i}$ from the standard normal distribution with equal correlation $0.2$, and then set
    \begin{align}
    Z_{1i} = \frac{\exp(V_i)-E[\exp(V_i)]}{\sqrt{\mathrm{Var}(\exp(V_i))}}~,
    \end{align}
    where $E[\exp(V_i)]=\exp(0.5)$ and $\sqrt{\mathrm{Var}(\exp(V_i))}=2.16$ (approximately).

(2) Independently, $\epsilon_i$ is generated from a mixture of a $N(-1/9,1)$ variable with probability $0.9$ and a $N(1,4)$ variable with probability $0.1$. When coding, you may first draw a sample $\eta_i$ from a Bernoulli distribution with success probability $0.9$, and then you draw $\epsilon_i$ from $N(-1/9,1)$ if $\eta_i=1$ and otherwise draw $\epsilon_i$ from $N(1,4)$. There may be a Python package for implementing this in a compact fashion, but, if not, I would just set $\epsilon_i=\eta_i W_{1i} + (1-\eta_i)W_{2i}$ for $W_{1i}\sim N(-1/9,1)$ and independently $W_{2i}\sim N(1,4)$.

(3) Generate $Y_i$ according to (1) for each pair $(\varrho_0,\lambda_0)$ to be specified.

Given $\{Y_i,Z_{1i},Z_{2i},Z_{3i}\}_{i=1}^n$, we then consider the following regression:
  \begin{align}
  Y_i=\alpha_0+\beta_1Z_{1i} + \beta_2 Z_{2i} + \beta_3 Z_{3i} + U_i~.
  \end{align}
Note that if $\varrho_0=0$, then $E[U_i|Z_{1i},Z_{2i},Z_{3i}]=0$ (the model is correctly specified), and if $\varrho_0\neq 0$, then $E[U_i|Z_{1i},Z_{2i},Z_{3i}]\neq 0$ so the model is misspecified (but one still has $E[U_i(1,Z_{1i},Z_{2i},Z_{3i})]=0$). The model exhibits conditional heteroskedasticity if $\lambda_0\neq 0$. In what follows, set $\lambda_0=1$ and $n=200$.

 You should manually generate the bootstrap samples and compute the bootstrap statistics, as described on p.51 of the lecture notes.

(a) Let $\varrho_0=0$. Compute the empirical rejection rates of the two-sided $t$-test for $\mathrm H_0: \beta_1=1$ at the significance level $\alpha=5\%$, based on the standard normal approximation, the pairs bootstrap and the wild bootstrap using Radamacher weights. Use $5000$ Monte Carlo replications and $200$ bootstrap repetitions. The closer the empirical rejection rates are to $\alpha$ (the nominal rate), the better the test performs. See the pdf of the problem set for a sketch of the coding structure.

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math as mt
from scipy.stats import bernoulli
import scipy.stats
from scipy.stats import bootstrap
import statsmodels.api as sm
from sympy.stats import Rademacher, sample

In [28]:
# The desired mean values of the sample.
n=200
mu = np.array([0.0, 0.0, 0.0])
    # The desired covariance matrix.
c = np.array([
        [  1, 0.2, 0.2],
        [ 0.2,  1,  0.2],
        [ 0.2,  0.2,  1]
     ])

In [30]:
def mysimul(r, b, n, rho, lam, B1):
    RejN=np.zeros([r,1])
    RejW=np.zeros([r,1])
    RejP=np.zeros([r,1])
    
    for i in range(r):
        # Generate the random samples.
        rng = np.random.default_rng()
        y = rng.multivariate_normal(mu, c, size=n)
        vi=y[:,0]
        z2i=y[:,1]
        z3i=y[:,2]
        a=np.exp(vi)
        z1i=(a-mt.exp(0.5))/2.16
        def rvs(p,size=1):
            rvs = np.array([])
            for i in range(0,size):
                if np.random.rand() <= p:
                    a=1
                    rvs = np.append(rvs,a)
                else:
                    a=0
                    rvs = np.append(rvs,a)
                return rvs
        ni = bernoulli.rvs(0.9,size=n) 
        w1i=np.random.normal(-1/9,1,n)
        w2i=np.random.normal(1,4,n)
        ei=(ni*w1i)+(1-ni)*w2i
        yi=z1i+z2i+z3i+rho*z1i*z2i+(1+lam*z1i)*ei
        X = pd.DataFrame({'z1': z1i, 'z2': z2i, 'z3': z3i })
        X = sm.add_constant(X)
        reg1 = sm.OLS(yi, X).fit()
        B1_hat= reg1.params[1]
        se= reg1.HC0_se[1]
        t =np.absolute((B1_hat - B1) / se)
    
        BootP0 = []
        for _ in range(b):
            my_samples = pd.DataFrame({'yi':yi,'z1': z1i, 'z2': z2i, 'z3': z3i })
            sample_new= my_samples.sample(n=200, frac=None, replace=True, weights=None, random_state=None, axis=None, ignore_index=False)
            y_star = sample_new['yi']
            X_star = sample_new[['z1', 'z2', 'z3']]
            X_star = sm.add_constant(X_star)
            reg_BP = sm.OLS(y_star, X_star).fit()
            B1_hat_BP= reg_BP.params[1]
            se_BP= reg_BP.HC0_se[1]
            tn_BP= (B1_hat_BP - B1_hat)/se_BP
            BootP0.append(tn_BP)
        BootP= np.transpose(np.absolute(np.array(BootP0)))
    
        BootW0=[]
        yw=yi-B1*z1i
        xw=pd.DataFrame({'z2':z2i, 'z3':z3i})
        xw=sm.add_constant(xw)
        reg2=sm.OLS(yw,xw).fit()
        Bt=np.array(reg2.params)
        ut=yi-np.matmul(xw,Bt)
        for _ in range(b):
            rad= Rademacher('W')
            wi = sample(rad, size= 200)
            u_star=ut*wi
            y2=np.matmul(xw,Bt)+z1i+u_star
            reg3=sm.OLS(y2,X).fit()
            B1_hat_BW=reg3.params[1]
            se_BW=reg3.HC0_se[1]
            tn_BW=(B1_hat_BW-B1)/se_BW
            BootW0.append(tn_BW)
        BootW= np.transpose(np.absolute(np.array(BootW0)))
    
        CriticalP= np.quantile(BootP,0.95)
        CriticalW= np.quantile(BootW,0.95)
    
        RejN[i] = 1*(t > 1.96)
        RejP[i] = 1*(t > CriticalP)
        RejW[i] = 1*(t > CriticalW)
   
    RejN_mean= np.mean(RejN)
    RejP_mean= np.mean(RejP)
    RejW_mean= np.mean(RejW)
    
    print("The average rejection rate based on the standard normal approximation is", RejN_mean)
    print("The average rejection rate based on the Pair bootstrap is", RejP_mean)
    print("The average rejection rate based on the Wild bootstrap is", RejW_mean)


In [31]:
mysimul(r=1000, b=200, n=200, rho=0, lam=1, B1=1)

The average rejection rate based on the standard normal approximation is 0.144
The average rejection rate based on the Pair bootstrap is 0.065
The average rejection rate based on the Wild bootstrap is 0.058


(b) Repeat part (a) by setting $\varrho_0\in\{-0.5,0.5\}$ (so the model is misspecified), but now consider the two-sided $t$ test for $\mathrm H_0: \beta_1=1+\varrho_0 b$ for $b=0.41$ (note that $1+\varrho_0 b$ is the slope parameter for $Z_{1i}$ in the best linear prediction of $Y_i$ given $1,Z_{1i},Z_{2i}$, and $Z_{3i}$).

In [32]:
mysimul(r=1000, b=200, n=200, rho=0.5, lam=1, B1=1.205)

The average rejection rate based on the standard normal approximation is 0.177
The average rejection rate based on the Pair bootstrap is 0.094
The average rejection rate based on the Wild bootstrap is 0.053


In [33]:
mysimul(r=1000, b=200, n=200, rho=-0.5, lam=1, B1=0.795)

The average rejection rate based on the standard normal approximation is 0.163
The average rejection rate based on the Pair bootstrap is 0.068
The average rejection rate based on the Wild bootstrap is 0.039
